In [1]:
# !pip install selenium

In [1]:
import urllib
import json
from bs4 import BeautifulSoup
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException

from selenium.webdriver.chrome.options import Options

import seaborn as sns
import sys
import time
import networkx as nx
import matplotlib.pyplot as plt
import re
from collections import defaultdict
%matplotlib inline

In [2]:
# selenium 속도향상위해 불필요한 옵션 사용x

options = Options()
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 
                                                    'plugins' : 2, 'popups': 2, 'geolocation': 2,
                                                    'notifications' : 2, 'auto_select_certificate': 2,
                                                    'fullscreen' : 2,
                                                    'mouselock' : 2, 'mixed_script': 2, 
                                                    'media_stream' : 2, 'media_stream_mic' : 2,
                                                    'media_stream_camera': 2, 'protocol_handlers' : 2,
                                                    'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2,
                                                    'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2,
                                                    'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2,
                                                    'durable_storage' : 2}}

options.add_experimental_option('prefs', prefs) 
options.add_argument("start-maximized") 
options.add_argument("disable-infobars") 
options.add_argument("--disable-extensions")

In [29]:
# brunch data crawling by 셀레니엄
# source reference : http://stackoverflow.com/questions/12519074/scrape-websites-with-infinite-scrolling

# category_list = ['지구한바퀴_세계여행?q=g','그림·웹툰?q=g','시사·이슈?q=g','IT_트렌드?q=g','사진·촬영?q=g','',
#                  '취향저격_영화_리뷰?q=g','오늘은_이런_책?q=g','뮤직_인사이드?q=g','글쓰기_코치?q=g','직장인_현실_조언?q=g'
#                  ,'스타트업_경험담?q=g','육아_이야기?q=g','요리·레시피?q=g','건강·운동?q=g','멘탈_관리_심리_탐구?q=g',
#                  '디자인_스토리?q=g','문화·예술?q=g','건축·설계?q=g','인문학·철학?q=g','쉽게_읽는_역사?q=g','우리집_반려동물?q=g',
#                  '멋진_캘리그래피?q=g','사랑·이별?q=g','감성_에세이?q=g']

category_list = ['지구한바퀴_세계여행?q=g','그림·웹툰?q=g','시사·이슈?q=g','IT_트렌드?q=g','사진·촬영?q=g']
idlist = []

def crawlBrunchLink(dir):
    ## html crawling
    chromedriver = 'C:/selenium/chromedriver.exe'
    driver = webdriver.Chrome(chromedriver)
    driver.implicitly_wait(30)
    base_url = "https://brunch.co.kr/keyword/{dir}".format(dir=dir)
    verificationErrors = []
    accept_next_alert = True
#     delay = 3
    driver.get(base_url)
    #driver.find_element_by_link_text("All").click()

    cnt = 0
    while True : # 스크롤 횟수
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);") # 페이지 무한 스크롤
        time.sleep(1)
        cnt = cnt + 1
        if cnt % 20 == 0:
            html_source = driver.page_source
            data = html_source.encode('utf-8')
            soup = BeautifulSoup(data, 'html.parser')
            pd = soup.find_all('span',{'class':'publish_time'})
            if (len(re.findall('Sep [\d]{1,2}\. 2018', pd[len(pd)-1].text)) != 0) or (len(re.findall('Aug [\d]{1,2}\. 2018', pd[len(pd)-1].text)) != 0) or (len(re.findall('Jul [\d]{1,2}\. 2018', pd[len(pd)-1].text)) != 0) or (len(re.findall('Jun [\d]{1,2}\. 2018', pd[len(pd)-1].text)) != 0):     
                break
            else : 
                pass

                    
    ## extract follower, following data
    html_source = driver.page_source
    classes = soup.find_all('li',{'class':'list_has_image animation_up'})

    for c in classes:
        follwing = c.get('data-articleuid')
        idlist.append(follwing.replace('_','/')) # 
    
#     driver.quit()
    driver.close()
    return idlist

In [30]:
writer_list = [] # 크롤링 해온 해당글 url 리스트

for dir in category_list:
    idlist =[] # idlist 누적방지
    crawlBrunchLink(dir)
    writer_list.append(idlist)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=77.0.3865.120)


In [8]:
# tmp_스타트업 =writer_list[0]
# tmp_육아 = writer_list[0]
# tmp_요리레시피 = writer_list[1]
# tmp_건강운동 = writer_list[2]
# tmp_멘탈관리 = writer_list[3]

In [7]:
len(writer_list[0])

4120

In [17]:
## 게시글 크롤링

import requests
import json

data = []
json_data = {}
# res_text = []

def def_craw(writer_list,input_catenum):
    global json_data
    global data
    res_text = []
    tag_keyword=[]
    
    tag_title,tag_nickname,tag_publish_date,tag_url,tag_url_plink = None,None,None,None,None
    tag_share,tag_like = str,str
    for i,url in enumerate(writer_list[input_catenum]):
        print("전체%d개의 게시글 중 %d번째 글 진행중" %(len(writer_list[input_catenum]),i+1))
        if res_text == []:
            pass
        else :
            json_data['title'] = tag_title.text  
            json_data['nickname'] = tag_nickname
            json_data['publish_date'] = tag_publish_date
            json_data['keyword'] = tmp_keyword   
            json_data['like'] = tag_like # like 없는 경우 ''
            json_data['share'] = tag_share # share 없는 경우 None
            json_data['comment'] = tag_comment # comment 없는 경우 ''
            json_data['url'] = tag_url
            json_data['url_plink'] = tag_url_plink 
            json_data['text'] = res_text

        data.append(json_data)
#         print(data)
        
        json_data = {} # 초기화
        tmp_keyword = [] # 초기화
        res_text = []
        
        # beautifulsoup 
        html = requests.get('https://brunch.co.kr/@@{text_url}'.format(text_url=url))
        soup = BeautifulSoup(html.text, 'html.parser')
#         tag_text_p = soup.find_all('p') # 해당페이지의 본문

        tag_title = soup.find('title') # 해당페이지의 Title
        tag_nickname = soup.find("meta",{'name':'article:media_name'})['content'] # 글쓴이 닉네임
        tag_url = soup.find("meta",property='og:url')['content'] # 본주소
        tag_url_plink = soup.find("meta",property='dg:plink')['content'] # 암호주소? # 모바일?
        tag_publish_date = soup.find("meta",property='article:published_time')['content'] # 본주소
        tag_keyword = soup.find_all('a',href=re.compile('/keyword')) # 해당 글의 키워드
        tag_like = soup.find('span',{'class':'f_l text_like_count text_default text_with_img_ico ico_likeit_like #like'}) #좋아요 수
        tag_share = soup.find('span',{'class':'f_l text_share_count text_default text_with_img_ico'}) # 공유 수
        tag_comment = soup.find('span',{'class':'f_l text_comment_count text_default text_with_img_ico'}) # 댓글 수
        tag_url_meta = soup.find("meta",property='ks:richscrap')['content']
        
        to_json = json.loads(tag_url_meta)
        for i in to_json['cards']:
            if i['type'] == 'text':
                res_text.append(i['body']['content'])

        if tag_like == None:
            tag_like = "0"
        else:
            tag_like = tag_like.text # 좋아요 수
            
        if tag_share == None:
            tag_share == "0"
        else:
            tag_share = tag_share.text # 공유 수
        
        if tag_comment == None:
            tag_comment =="0"
        else:
            tag_comment = tag_comment.text
#         for text in tag_text:
#             tmp_text.append(text.text) # p태그 text

        for keyword in tag_keyword:
            tmp_keyword.append(keyword.text)

In [ ]:
# 지구한바퀴_세계여행?q=g => 0
# 그림·웹툰?q=g => 1
# 시사·이슈?q=g => 2
# IT_트렌드?q=g => 3
# 사진·촬영?q=g => 4
# 취향저격_영화_리뷰?q=g => 5
# 오늘은_이런_책?q=g => 6
# 뮤직_인사이드?q=g => 7
# 글쓰기_코치?q=g => 8
# 직장인_현실_조언?q=g => 9
# 스타트업_경험담?q=g => 10
# 육아_이야기?q=g => 11
# 요리·레시피?q=g => 12
# 건강·운동?q=g => 13
# 멘탈_관리_심리_탐구?q=g => 14
# 디자인_스토리?q=g => 15
# 문화·예술?q=g => 16
# 건축·설계?q=g => 17
# 인문학·철학?q=g' => 18
# 쉽게_읽는_역사?q=g => 19
# 우리집_반려동물?q=g => 20
# 멋진_캘리그래피?q=g => 21
# 사랑·이별?q=g => 22
# 감성_에세이?q=g => 23

In [18]:
# 실행
# def_craw(writer_list, "실행시킬 카테고리 Num")
def_craw(writer_list,0)

IndexError: list index out of range

In [107]:
len(data)

14

In [131]:
data

[{},
 {'title': '아직 떠나고 싶지 않아',
  'nickname': '아라',
  'publish_date': '2019-10-21T11:00+0900',
  'keyword': ['해외여행', '피렌체', '여행에세이'],
  'like': '1',
  'share': None,
  'comment': '',
  'url': 'https://brunch.co.kr/@dugid84/20',
  'url_plink': 'https://brunch.co.kr/@@7qGP/20',
  'text': ['     피렌체 도심으로 들어가기 전 가까운 주유소에서 기름을 넣었다. 기름을 가득 채워서 반납하게 되어있는데 대체 왜 [가득] 버튼이 없단 말인가! 기름이 어느 정도 들어가는지에 대한 감이 전혀 없는지라 일단 10유로를 주유했더니 아직 한 칸 아래다. 그다음 단위는 5유로인데 돌려받지도 못할 돈을 넣고 반도 안 넣은 것 같은데 (내 이럴 줄 알았지) 기름이 가득 찼단다. &lsquo;이렇게 돌려주지 않는 돈만 해도 꽤 쏠쏠할지도?&rsquo; 싶으면서 조금 억한 심정이 생긴다.',
   ' ',
   '     분명 [가득] 버튼도 만들 수 있고 현금 반환 기능도 탑재할 수 있을 것 같은데 선지불과 해당량의 기름을 주입하는 기능밖에 없는 주유기가 원망스럽다. 어떤 분야에서건 조금 더 편하게 조금 더 빠르게를 추구하는 한국의 생활에 익숙해진 나는 쉽게 불편함을 느낀다. (편하고 빠름이 결국 수익으로 나타나기 때문이겠지만) 역시 나는 잘 적응된 소비 원숭이라는 것을 다시 한번 확인했다.',
   ' ',
   '     ZTL(Zona a traffico limitato, 법적으로 정해 놓은 교통 제한 구역)에 덜컥 들어가 단속카메라에 찍히는 불상사 없이 차를 제대로 반납하려면 &lsquo;다리를 건너 좌회전! 좌회전!&rsquo;을 되뇌며 피렌체에 도착했다. 갑자기 늘어난 차와 자전거에 조금 놀랐지만 &lsquo;성공적인&rsquo; 좌회전으로 렌터카 

In [29]:
import json
from collections import OrderedDict

# Ready for data
# OrderDict => 딕셔너리 key 자동정렬 방지
del data[0] # 위 코드상으로 data 맨앞에 빈 dict들어감으로 여기서 빈 dict 삭제
to_json = OrderedDict() 
full_data = OrderedDict()

to_json['name'] = "지구한바퀴_세계여행" # 카테고리명으로 지정 
to_json['version'] = "2019-10-20" # 날짜
to_json['data'] = data

# Write JSON
# json 저장 부분. 이름 바꿔서 저장해야함
# brunch_json_category_날짜+버전 형식으로 저장할것
with open('bruch_json_0_1910201', 'w', encoding="utf-8") as make_file: 
    json.dump(to_json, make_file, ensure_ascii=False, indent="\t")

UnicodeEncodeError: 'utf-8' codec can't encode characters in position 41-42: surrogates not allowed

In [241]:
## json_to_dataframe

import json

json_obj = pd.read_json('bruch_json_0_1910201')
# result = json.loads(json_obj)

# result_df = pd.DataFrame(json_obj['data'],columns=['title','nickname','url','url_plink','publish_data','keyword','like','share','comment' ])


,name,version,data
0,吏�援ы븳諛뷀��_�꽭怨꾩뿬�뻾,2019-10-20,"{'title': '洹몃읆�뿉�룄 遺덇뎄�븯怨� �뙣�뀡�뵒�옄�씠�꼫 2.', '..."
1,吏�援ы븳諛뷀��_�꽭怨꾩뿬�뻾,2019-10-20,"{'title': '�꽭怨꾩뿬�뻾, 洹� 泥ル궇.', 'nickname': '�뵾�..."
2,吏�援ы븳諛뷀��_�꽭怨꾩뿬�뻾,2019-10-20,"{'title': 'Rinuccio 1180 ����씠�꼫由� �젅�뒪�넗�옉', ..."
3,吏�援ы븳諛뷀��_�꽭怨꾩뿬�뻾,2019-10-20,{'title': '�궗�엺�뱾�씠 �슦由� �궓留� 蹂닿퀬 �듅�씠�븳�뜲 硫뗭엳...
4,吏�援ы븳諛뷀��_�꽭怨꾩뿬�뻾,2019-10-20,"{'title': '�븳援��뿉 媛�硫� �겢�옒�떇�쓣 �뱽寃� �맆源�?', '..."
...,...,...,...
342,吏�援ы븳諛뷀��_�꽭怨꾩뿬�뻾,2019-10-20,"{'title': 'Matias', 'nickname': '�떎�쁺', 'publi..."
343,吏�援ы븳諛뷀��_�꽭怨꾩뿬�뻾,2019-10-20,{'title': '�뼱�뵒濡� 媛��떆�굹�슂? �뿬�뻾�옄媛� 紐⑥씠�뒗 怨�....
344,吏�援ы븳諛뷀��_�꽭怨꾩뿬�뻾,2019-10-20,{'title': '�슦由ш�� 紐곕옄�뜕 泥댁퐫 紐⑤씪鍮꾩븘 �씠�빞湲� - 2�...
345,吏�援ы븳諛뷀��_�꽭怨꾩뿬�뻾,2019-10-20,"{'title': '�뿬�뱺 �궡�쓽 硫뗭쭊 洹몃���뿉寃�', 'nickname'..."


In [60]:
# html = requests.get('https://brunch.co.kr/@{text_url}'.format(text_url='@3799/35'))
# soup = BeautifulSoup(html.text, 'html.parser')
# tag_title = soup.find('title') # 해당페이지의 Title
# tag_nickname = soup.find("meta",{'name':'article:media_name'})['content'] # 글쓴이 닉네임
# tag_url = soup.find("meta",property='og:url')['content'] # 본주소
# tag_url_plink = soup.find("meta",property='dg:plink')['content'] # 암호주소? # 모바일?
# tag_publish_date = soup.find("meta",property='article:published_time')['content'] # 본주소
# tag_keyword = soup.find_all('a',href=re.compile('/keyword')) # 해당 글의 키워드
# tag_like = soup.find('span',{'class':'f_l text_like_count text_default text_with_img_ico ico_likeit_like #like'}) # 좋아요 수
# tag_share = soup.find('span',{'class':'f_l text_share_count text_default text_with_img_ico'}) # 공유 수
# tag_comment = soup.find('span',{'class':'f_l text_comment_count text_default text_with_img_ico'}) # 댓글 수는 이미지임

# tag_url_meta = soup.find("meta",property='ks:richscrap')['content']
